## L2 DataFrame APIs (Transformations)

In [111]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F

In [112]:
spark = SparkSession \
    .builder \
    .appName("Spark Training - DF APIs") \
    .getOrCreate()

In [7]:
ord = spark.read.load('PracticeFiles/Orders', sep=',', format='csv', schema=('order_id int,order_date timestamp, order_customer_id int, order_status string'))
ord.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [9]:
data = [('Robert', 35, 40, 40), ('Robert', 35, 40, 40),('Ram', 31, 33, 29),('Ram', 31, 33, 29)]
emp = spark.createDataFrame(data=data, schema=['name', 'score1','score2', 'score3'])
emp.show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    29|
+------+------+------+------+



## 1. select API

- For selecting one or more columns
- Allows you apply functions on selected columns

### 1.1. Selecting one/multiple columns

In [11]:
# 2 methods of using it as outlined in script below
ord.select(ord.order_id, 'order_id', (ord.order_id + 10).alias('order10')).show(5)

+--------+--------+-------+
|order_id|order_id|order10|
+--------+--------+-------+
|       1|       1|     11|
|       2|       2|     12|
|       3|       3|     13|
|       4|       4|     14|
|       5|       5|     15|
+--------+--------+-------+
only showing top 5 rows



### 1.2 Applying functions on selected columns

In [13]:
ord.select('order_status',F.lower(ord.order_status)).show(5)

+---------------+-------------------+
|   order_status|lower(order_status)|
+---------------+-------------------+
|         CLOSED|             closed|
|PENDING_PAYMENT|    pending_payment|
|       COMPLETE|           complete|
|         CLOSED|             closed|
|       COMPLETE|           complete|
+---------------+-------------------+
only showing top 5 rows



## 2. selectExpr()

- A variant of the selct api that accepst SQL expressions
- Advantage: Comes in handy when you want to perform an operation that has no built-in Spark functions

In [14]:
ord.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [20]:
# first using normal select api
ord.select(F.substring(ord.order_date, 1, 4).alias('order_year')).show(5)

+----------+
|order_year|
+----------+
|      2013|
|      2013|
|      2013|
|      2013|
|      2013|
+----------+
only showing top 5 rows



In [23]:
# then using selectExpr
ord.selectExpr("substring(order_date, 1,4) as order_year").show(5)

+----------+
|order_year|
+----------+
|      2013|
|      2013|
|      2013|
|      2013|
|      2013|
+----------+
only showing top 5 rows



## 3. withColumn(colName, col)
- Applies transformation to only selected columns
- First arg is the alias name. If we give an alias name same as the column name, the transformation will be applied on the same column
- otherwise a new column will be formed. Avoid giving an existing column name as alias

In [25]:
ord.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [28]:
# lets create a new columns called order_year
ord.withColumn('order_year', F.substring(ord.order_date, 1,4)).show(5)

+--------+-------------------+-----------------+---------------+----------+
|order_id|         order_date|order_customer_id|   order_status|order_year|
+--------+-------------------+-----------------+---------------+----------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|      2013|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|      2013|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|      2013|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|      2013|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|      2013|
+--------+-------------------+-----------------+---------------+----------+
only showing top 5 rows



In [27]:
# lets modify existing column order_date
ord.withColumn('order_date', F.substring(ord.order_date, 1,4)).show(5)

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|       1|      2013|            11599|         CLOSED|
|       2|      2013|              256|PENDING_PAYMENT|
|       3|      2013|            12111|       COMPLETE|
|       4|      2013|             8827|         CLOSED|
|       5|      2013|            11318|       COMPLETE|
+--------+----------+-----------------+---------------+
only showing top 5 rows



## 4. withColumnRenamed(existingCol, newCol)

Renames and existing columns

In [30]:
ord.withColumnRenamed('order_id', 'order_id1').show(5)

+---------+-------------------+-----------------+---------------+
|order_id1|         order_date|order_customer_id|   order_status|
+---------+-------------------+-----------------+---------------+
|        1|2013-07-25 00:00:00|            11599|         CLOSED|
|        2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|        3|2013-07-25 00:00:00|            12111|       COMPLETE|
|        4|2013-07-25 00:00:00|             8827|         CLOSED|
|        5|2013-07-25 00:00:00|            11318|       COMPLETE|
+---------+-------------------+-----------------+---------------+
only showing top 5 rows



## 5. drop(*cols)

- Takes one or more columns as argument and drops them
- Do not pass in a list except you unpack with *

In [32]:
ord_new = ord.drop('order', 'order_date')
ord_new.show(5)

+--------+-----------------+---------------+
|order_id|order_customer_id|   order_status|
+--------+-----------------+---------------+
|       1|            11599|         CLOSED|
|       2|              256|PENDING_PAYMENT|
|       3|            12111|       COMPLETE|
|       4|             8827|         CLOSED|
|       5|            11318|       COMPLETE|
+--------+-----------------+---------------+
only showing top 5 rows



## 6. dropDuplicates(subset=None)
- Drop duplicate rows.
- Optionally can consider only subset of columns

In [34]:
# lets inspect the duplicates
emp.show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    29|
+------+------+------+------+



In [35]:
# let's drop duplicate rows
emp.dropDuplicates().show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
+------+------+------+------+



In [37]:
# lets drop rows with duplicate name - score1 pairs
emp.dropDuplicates(subset = ['name','score1']).show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|   Ram|    31|    33|    29|
|Robert|    35|    40|    40|
+------+------+------+------+



## 7. Filter: (also where)
- Already visited - see lesson 1
- use `&` for 'and' and `|` for 'or'
- use column function `isin` for multiple search
- use IN operator for sql style syntax

In [40]:
# normal syntax vs sql syntax
ord.where(ord.order_status.isin('COMPLETE','CLOSED')).show(5)

+--------+-------------------+-----------------+------------+
|order_id|         order_date|order_customer_id|order_status|
+--------+-------------------+-----------------+------------+
|       1|2013-07-25 00:00:00|            11599|      CLOSED|
|       3|2013-07-25 00:00:00|            12111|    COMPLETE|
|       4|2013-07-25 00:00:00|             8827|      CLOSED|
|       5|2013-07-25 00:00:00|            11318|    COMPLETE|
|       6|2013-07-25 00:00:00|             7130|    COMPLETE|
+--------+-------------------+-----------------+------------+
only showing top 5 rows



In [39]:
# sql-like syntax as we've already visited the others
ord.where("order_status in ('COMPLETE','CLOSED')").show(5)

+--------+-------------------+-----------------+------------+
|order_id|         order_date|order_customer_id|order_status|
+--------+-------------------+-----------------+------------+
|       1|2013-07-25 00:00:00|            11599|      CLOSED|
|       3|2013-07-25 00:00:00|            12111|    COMPLETE|
|       4|2013-07-25 00:00:00|             8827|      CLOSED|
|       5|2013-07-25 00:00:00|            11318|    COMPLETE|
|       6|2013-07-25 00:00:00|             7130|    COMPLETE|
+--------+-------------------+-----------------+------------+
only showing top 5 rows



## 8. sort() or orderBy() APIs
- very costly operations

In [41]:
# method 1
ord.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [43]:
# sort by order_customer_id
ord.sort(ord.order_customer_id.asc()).show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|   22945|2013-12-13 00:00:00|                1|       COMPLETE|
|   57963|2013-08-02 00:00:00|                2|        ON_HOLD|
|   15192|2013-10-29 00:00:00|                2|PENDING_PAYMENT|
|   67863|2013-11-30 00:00:00|                2|       COMPLETE|
|   33865|2014-02-18 00:00:00|                2|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [44]:
ord.sort(ord.order_date.desc(), ord.order_customer_id.asc()).show(5)

+--------+-------------------+-----------------+------------+
|order_id|         order_date|order_customer_id|order_status|
+--------+-------------------+-----------------+------------+
|   57617|2014-07-24 00:00:00|                3|    COMPLETE|
|   57733|2014-07-24 00:00:00|               17|      CLOSED|
|   57684|2014-07-24 00:00:00|               98|    COMPLETE|
|   57695|2014-07-24 00:00:00|              112|    COMPLETE|
|   57598|2014-07-24 00:00:00|              138|     PENDING|
+--------+-------------------+-----------------+------------+
only showing top 5 rows



In [45]:
# method 2: using the ascending= argument
ord.sort(ord.order_date, ord.order_customer_id, ascending= [0, 1]).show(5)

+--------+-------------------+-----------------+------------+
|order_id|         order_date|order_customer_id|order_status|
+--------+-------------------+-----------------+------------+
|   57617|2014-07-24 00:00:00|                3|    COMPLETE|
|   57733|2014-07-24 00:00:00|               17|      CLOSED|
|   57684|2014-07-24 00:00:00|               98|    COMPLETE|
|   57695|2014-07-24 00:00:00|              112|    COMPLETE|
|   57598|2014-07-24 00:00:00|              138|     PENDING|
+--------+-------------------+-----------------+------------+
only showing top 5 rows



In [46]:
# or using pass True/False in ascending=
ord.sort(ord.order_date, ord.order_customer_id, ascending= [False, True]).show(5)

+--------+-------------------+-----------------+------------+
|order_id|         order_date|order_customer_id|order_status|
+--------+-------------------+-----------------+------------+
|   57617|2014-07-24 00:00:00|                3|    COMPLETE|
|   57733|2014-07-24 00:00:00|               17|      CLOSED|
|   57684|2014-07-24 00:00:00|               98|    COMPLETE|
|   57695|2014-07-24 00:00:00|              112|    COMPLETE|
|   57598|2014-07-24 00:00:00|              138|     PENDING|
+--------+-------------------+-----------------+------------+
only showing top 5 rows



## 9. sortWithinPartitions
Used to avoid sorting globally (like above), but rather sort within a group/partition.

In [68]:
data = [('a',1), ('d',4), ('c', 3), ('b', 2), ('e', 5), ('f',10)]
df = spark.createDataFrame(data=data, schema='col1 string, col2 int')
df.show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   d|   4|
|   c|   3|
|   b|   2|
|   e|   5|
|   f|  10|
+----+----+



In [72]:
# let's repartition to 3 partitions
df = df.repartition(3)

In [73]:
# how many partitions are there in the data
df.rdd.getNumPartitions()

3

In [74]:
# how's the data distributed in these 8 partitions
df.rdd.glom().collect()

[[Row(col1='c', col2=3), Row(col1='b', col2=2)],
 [Row(col1='a', col2=1)],
 [Row(col1='d', col2=4), Row(col1='e', col2=5), Row(col1='f', col2=10)]]

In [75]:
# using normal sort
df.sort(df.col1.asc()).show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   c|   3|
|   d|   4|
|   e|   5|
|   f|  10|
+----+----+



This has sorted it globally, i.e. every value of col1 has been sorted

In [76]:
# using sortWithinPartition
df.sortWithinPartitions(df.col1.asc(), ).show()

+----+----+
|col1|col2|
+----+----+
|   b|   2|
|   c|   3|
|   a|   1|
|   d|   4|
|   e|   5|
|   f|  10|
+----+----+



So now if you extract this data into 3 files, each of those 3 files will be sorted

## 10. Set Operator APIs
- union() and unionAll()
- unionByName()
- intersect() and intersectAll()
- exceptAll()

#### 10.1 union() and unionAll()
- Behave in same manner, retains duplicates

In [108]:
# create df
df1 = spark.range(10)
df2 = spark.range(5,15)
df1.show(), df2.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



(None, None)

`notice:` 5 - 9 are common to both dataframes

In [85]:
df1.union(df2).show()
# `Notice:` 5 - 9 repeat once

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



#### 10.2 unionByName()
- This function differst to the union in that it resolves functions by name (not by position)

In [95]:
data1 = [('a',1), ('b', 2), ('d',4), ('c', 3)]
data2 = [(2, 'b'), (5, 'e'), (10, 'f')]
df1 = spark.createDataFrame(data=data1, schema='col1 string, col2 int')
df2 = spark.createDataFrame(data=data2, schema='col2 int, col1 string')
df1.show(), df2.show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   d|   4|
|   c|   3|
+----+----+

+----+----+
|col2|col1|
+----+----+
|   2|   b|
|   5|   e|
|  10|   f|
+----+----+



(None, None)

notice ordering of columns for 
- df1: col1 and col2
- df2: col2 and col1
    

In [96]:
# union() combines by column positions and not column names
df1.union(df2).show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   d|   4|
|   c|   3|
|   2|   b|
|   5|   e|
|  10|   f|
+----+----+



In [99]:
# in these situations, we should be using unionByName()
df1.unionByName(df2).show()
# However notice distinct are retained e.g. b, 2. Use distinct() to dedup

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   d|   4|
|   c|   3|
|   b|   2|
|   e|   5|
|   f|  10|
+----+----+



#### 10.3 intersect() and intersectAll()

- intersect() should remove duplicates while intersectAll() doesnt

In [102]:
# create df
df1 = spark.range(10)
df2 = spark.range(5,15)
df1.show(), df2.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



(None, None)

In [103]:
df1.intersect(df2).show()

+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [104]:
df1.intersectAll(df2).show()

+---+
| id|
+---+
|  7|
|  6|
|  9|
|  5|
|  8|
+---+



#### 10.4 exceptAll()
- Returns rows present in base dataframe but not in the another

In [110]:
df1.exceptAll(df2).show()

+---+
| id|
+---+
|  0|
|  1|
|  3|
|  2|
|  4|
+---+



## 11. Join APIs

In [113]:
df1 = spark.createDataFrame(data= [(1, 'Robert'),(2, 'Ria'), (3, 'James')], schema='empid int, empname string')
df2 = spark.createDataFrame(data= [(2, 'USA'),(4, 'India')], schema = 'empid int, country string')

df1.show(), df2.show()

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    2|    Ria|
|    3|  James|
+-----+-------+

+-----+-------+
|empid|country|
+-----+-------+
|    2|    USA|
|    4|  India|
+-----+-------+



(None, None)

#### 11.1 inner join
- for multi column joins, on = (condition1) & (condition2)

In [150]:
# inner join = the default
# for multi column joins, on = (condition1) & (condition2)
df1.join(df2, df1.empid == df2.empid).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    2|    Ria|    2|    USA|
+-----+-------+-----+-------+



In [117]:
# inner join, selecting just a couple of columns
df1.join(df2, df1.empid == df2.empid).select(df1.empid, df1.empname, df2.country).show()

+-----+-------+-------+
|empid|empname|country|
+-----+-------+-------+
|    2|    Ria|    USA|
+-----+-------+-------+



#### 11.2 Left outer (similar syntax for right join)

In [118]:
df1.join(df2, df1.empid == df2.empid, how='left').select(df1.empid, df1.empname, df2.country).show()

+-----+-------+-------+
|empid|empname|country|
+-----+-------+-------+
|    1| Robert|   null|
|    3|  James|   null|
|    2|    Ria|    USA|
+-----+-------+-------+



### 11.3 Full join

- Get all records from both tables

In [120]:
df1.join(df2, df1.empid == df2.empid, how='full').select(df1.empid, df1.empname, df2.country).show()

+-----+-------+-------+
|empid|empname|country|
+-----+-------+-------+
|    1| Robert|   null|
|    3|  James|   null|
| null|   null|  India|
|    2|    Ria|    USA|
+-----+-------+-------+



#### 11.4 left anti

- Returns records present in left table but not in the right table
- `Note:` With this join, you cannot select a column from the right dataframe. You can only select from the left one. This is because it only checks the left df

In [132]:
df1.join(df2, df1.empid == df2.empid, how='left_anti').show()

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    3|  James|
+-----+-------+



#### 11.5 left semi
- Similar to inner join but much faster/efficient
- `Difference:` this only broadcasts/checks the columns in the left dataframe. So you cannot retrieve any columns from right dataframe
- `When to use it:` If you dont require a column from the right table

In [133]:
df1.show(), df2.show()

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    2|    Ria|
|    3|  James|
+-----+-------+

+-----+-------+
|empid|country|
+-----+-------+
|    2|    USA|
|    4|  India|
+-----+-------+



(None, None)

In [135]:
df1.join(df2, df1.empid == df2.empid, how='leftsemi').show()

+-----+-------+
|empid|empname|
+-----+-------+
|    2|    Ria|
+-----+-------+



#### 11.6 self join
- Not an API
- Just to show how to perform it in pyspark

In [138]:
df = spark.createDataFrame(data= [(1, 'Robert', 2),(2, 'Ria', 3), (3, 'James', 5)], schema='empid int, empname string, managerid int')
df.show()

+-----+-------+---------+
|empid|empname|managerid|
+-----+-------+---------+
|    1| Robert|        2|
|    2|    Ria|        3|
|    3|  James|        5|
+-----+-------+---------+



In [149]:
# let's get manager names - note: if you dont use the F.col function, spark throws an error
df.alias('df_left')\
    .join(df.alias('df_right'), F.col('df_left.empid')==F.col('df_right.managerid'), 'inner')\
    .select(F.col('df_left.empid'), F.col('df_left.empname'), F.col('df_right.managerid'), F.col('df_right.empname').alias('manager_name'))\
    .show()

+-----+-------+---------+------------+
|empid|empname|managerid|manager_name|
+-----+-------+---------+------------+
|    3|  James|        3|         Ria|
|    2|    Ria|        2|      Robert|
+-----+-------+---------+------------+



## 12. Aggregation APIs

In [153]:
orderItem = spark.read.load('PracticeFiles/Order_items', sep=',', format='csv', schema='order_item_id int, order_item_order_id int, order_item_proeuct_id int, quantity int, subtotal float, price float')
orderItem.show(5)

+-------------+-------------------+---------------------+--------+--------+------+
|order_item_id|order_item_order_id|order_item_proeuct_id|quantity|subtotal| price|
+-------------+-------------------+---------------------+--------+--------+------+
|            1|                  1|                  957|       1|  299.98|299.98|
|            2|                  2|                 1073|       1|  199.99|199.99|
|            3|                  2|                  502|       5|   250.0|  50.0|
|            4|                  2|                  403|       1|  129.99|129.99|
|            5|                  4|                  897|       2|   49.98| 24.99|
+-------------+-------------------+---------------------+--------+--------+------+
only showing top 5 rows



#### 12.1 summary function

- Displays basic summary functions for all numeric fields i.e. count, mean, stddev, min, 25, 50, 75 and max percentiles

In [155]:
# for all numeric fields
orderItem.summary().show()

+-------+-----------------+-------------------+---------------------+------------------+------------------+------------------+
|summary|    order_item_id|order_item_order_id|order_item_proeuct_id|          quantity|          subtotal|             price|
+-------+-----------------+-------------------+---------------------+------------------+------------------+------------------+
|  count|           172198|             172198|               172198|            172198|            172198|            172198|
|   mean|          86099.5|  34442.56682423721|    660.4877176273824|2.1821275508426345|199.32066922046081|133.75906959048717|
| stddev|49709.42516431533| 19883.325171992343|     310.514472790008|1.4663523175387134|112.74303987146804|118.55893633258484|
|    min|                1|                  1|                   19|                 1|              9.99|              9.99|
|    25%|            43033|              17204|                  403|                 1|            119.98|    

In [159]:
# for only a subset of fields
orderItem.select(orderItem.quantity, orderItem.subtotal, orderItem.price).summary().show()

+-------+------------------+------------------+------------------+
|summary|          quantity|          subtotal|             price|
+-------+------------------+------------------+------------------+
|  count|            172198|            172198|            172198|
|   mean|2.1821275508426345|199.32066922046081|133.75906959048717|
| stddev|1.4663523175387134|112.74303987146804|118.55893633258484|
|    min|                 1|              9.99|              9.99|
|    25%|                 1|            119.98|              50.0|
|    50%|                 1|            199.92|             59.99|
|    75%|                 3|            299.95|            199.99|
|    max|                 5|           1999.99|           1999.99|
+-------+------------------+------------------+------------------+



#### 12.2 average, min, max, median etc
- Available from pyspark.functions

In [161]:
orderItem.select(F.avg(orderItem.price)).show()

+------------------+
|        avg(price)|
+------------------+
|133.75906959048717|
+------------------+



In [164]:
# round and rename field
orderItem.select(F.round(F.avg(orderItem.price), 2).alias('avg_price')).show()

+---------+
|avg_price|
+---------+
|   133.76|
+---------+



#### 12.3 sum and sumDistinct
- sum just adds up everything
- sumDistinct first removes duplicates before summing

In [166]:
orderItem.select(F.round(F.sum(orderItem.price),2), F.round(F.sumDistinct(orderItem.price),2)).show()

+--------------------+-----------------------------+
|round(sum(price), 2)|round(sum(DISTINCT price), 2)|
+--------------------+-----------------------------+
|       2.303304427E7|                      9832.42|
+--------------------+-----------------------------+



#### 12.4 count and countDistinct
- similar explanation as above

In [167]:
orderItem.select(F.count(orderItem.price), F.countDistinct(orderItem.price)).show()

+------------+---------------------+
|count(price)|count(DISTINCT price)|
+------------+---------------------+
|      172198|                   57|
+------------+---------------------+



#### 12.5 first and last
- min and max can be used in their place expecially for numeric values

In [171]:
# inspect data
orderItem.sort(orderItem.price.desc()).show(5)

+-------------+-------------------+---------------------+--------+--------+-------+
|order_item_id|order_item_order_id|order_item_proeuct_id|quantity|subtotal|  price|
+-------------+-------------------+---------------------+--------+--------+-------+
|       171961|              68778|                  208|       1| 1999.99|1999.99|
|       172129|              68848|                  208|       1| 1999.99|1999.99|
|       172019|              68806|                  208|       1| 1999.99|1999.99|
|       171765|              68703|                  208|       1| 1999.99|1999.99|
|       172032|              68809|                  208|       1| 1999.99|1999.99|
+-------------+-------------------+---------------------+--------+--------+-------+
only showing top 5 rows



In [174]:
# let's get highest price
orderItem.sort(orderItem.price.desc()).select(F.first(orderItem.price)).show(1)

+------------+
|first(price)|
+------------+
|     1999.99|
+------------+



#### 12.6 collect_set, collect_list
- does what the names say. Sets contain unique elements while lists dont

In [175]:
df = spark.createDataFrame([(1,100),(2,150),(3,200),(4,50),(5,50)], schema='id int, salary int')
df.show()

+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   150|
|  3|   200|
|  4|    50|
|  5|    50|
+---+------+



In [181]:
# show(truncate=False) shows every element in field
df.select(F.collect_list(df.salary).alias('list'), F.collect_set(df.salary).alias('set')).show(truncate=False)

+-----------------------+-------------------+
|list                   |set                |
+-----------------------+-------------------+
|[100, 150, 200, 50, 50]|[150, 100, 50, 200]|
+-----------------------+-------------------+



In [185]:
# lets access first and second distinct salary
df.select(F.collect_list(df.salary)[1].alias('first_sal'), F.collect_set(df.salary)[2].alias('second_sal')).show()

+---------+----------+
|first_sal|second_sal|
+---------+----------+
|      150|        50|
+---------+----------+



#### 12.7 skewness, variance and stddev

In [187]:
df.select(F.skewness(df.salary).alias('skewness_sal'), F.variance(df.salary).alias('var_sal'), F.stddev(df.salary).alias('stdev_sal')).show()

+------------------+-------+-----------------+
|      skewness_sal|var_sal|        stdev_sal|
+------------------+-------+-----------------+
|0.3631734744194305| 4250.0|65.19202405202648|
+------------------+-------+-----------------+



## 13 GroupBy API
The functions below can be used to create GroupedData object:
- avg(), count(), min(), max(), sum(), agg(), pivot(), apply()


In [188]:
data = [
    ('James', 'Sales', 'NY', 900, 34),
    ('Robert', 'Sales', 'CA', 8100, 30),
    ('Lisa', 'Finance', 'CA', 9000, 24),
    ('Deja', 'Finance', 'CA', 9900, 40),
    ('Sugie', 'Finance', 'NY', 8300, 36),
    ('Ram', 'Finance', 'NY', 7900, 53),
    ('Kyle', 'Marketing', 'CA', 8000, 25),
    ('Reid', 'Marketing', 'NY', 9100, 50)
]

schema = ['empname', 'dept', 'state', 'salary', 'age']
df = spark.createDataFrame(data=data, schema=schema)

In [190]:
df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|   900| 34|
| Robert|    Sales|   CA|  8100| 30|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



In [192]:
# this creates a groupedData object
df.groupBy(df.dept)
# try help(df.groupBy(df.dept))

#### 13.1. Apply one function on one column

In [197]:
# apply functions on single column
df.groupBy('dept').sum('salary').show()

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|    Sales|       9000|
|  Finance|      35100|
|Marketing|      17100|
+---------+-----------+



#### 13.2 apply one function on multiple columns

In [198]:
df.groupBy('dept').min('salary','age').show()

+---------+-----------+--------+
|     dept|min(salary)|min(age)|
+---------+-----------+--------+
|    Sales|        900|      30|
|  Finance|       7900|      24|
|Marketing|       8000|      25|
+---------+-----------+--------+



### 13.3 Apply multiple aggregation functions
- use agg()

In [199]:
df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|   900| 34|
| Robert|    Sales|   CA|  8100| 30|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



In [202]:
# df.groupBy(df.dept).agg(F.avg(df.salary), F.max(df.age)).show() or
df.groupBy('dept').agg(F.avg('salary'), F.max('age'), F.min('salary')).show() 

+---------+-----------+--------+-----------+
|     dept|avg(salary)|max(age)|min(salary)|
+---------+-----------+--------+-----------+
|    Sales|     4500.0|      34|        900|
|  Finance|     8775.0|      53|       7900|
|Marketing|     8550.0|      50|       8000|
+---------+-----------+--------+-----------+



#### 13.4 Apply aggregations & use filter() or where()
- can be applied before grouping and aggregating as in example 1 below OR
- can be applied after grouping and aggregating as in example 2. Using and alias in this case really tidies up the columns

In [209]:
# example 1
df.filter(df.dept=='Finance')\
    .groupBy('state')\
    .agg(F.avg('salary'), F.max('age'), F.min('salary'))\
    .show()

+-----+-----------+--------+-----------+
|state|avg(salary)|max(age)|min(salary)|
+-----+-----------+--------+-----------+
|   CA|     9450.0|      40|       9000|
|   NY|     8100.0|      53|       7900|
+-----+-----------+--------+-----------+



In [213]:
# method 2 - use F.col on aliased columns
df.groupBy('dept')\
    .agg(F.avg('salary').alias('avg_sal'), F.max('age').alias('max_sal'), F.min('salary').alias('min_sal'))\
    .where(F.col('min_sal')>7_000)\
    .show() 

+---------+-------+-------+-------+
|     dept|avg_sal|max_sal|min_sal|
+---------+-------+-------+-------+
|  Finance| 8775.0|     53|   7900|
|Marketing| 8550.0|     50|   8000|
+---------+-------+-------+-------+



#### 13.5 Using pivot()
- Transpose rows into columns

In [214]:
df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|   900| 34|
| Robert|    Sales|   CA|  8100| 30|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



In [222]:
df1 = df.select('dept','state','salary')
df1

DataFrame[dept: string, state: string, salary: bigint]

In [237]:
# first we group it
df2 = df1.groupBy('dept','state').agg(F.sum('salary').alias('sum_salary'))
df2.show()

+---------+-----+----------+
|     dept|state|sum_salary|
+---------+-----+----------+
|  Finance|   NY|     16200|
|Marketing|   NY|      9100|
|    Sales|   CA|      8100|
|Marketing|   CA|      8000|
|  Finance|   CA|     18900|
|    Sales|   NY|       900|
+---------+-----+----------+



Goal with pivoting:
- to transpose state

In [248]:
# then transpose
df_t = df2.groupBy('dept')\
            .pivot('state')\
            .sum('sum_salary')

df_t.show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|  900|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



### 13.6 How to unpivot
In spark, there is no such function as unpivot. We can do it using stack()  function in selectExpr

Stack function accepts 2 argumants:
- n: the number of rows. How many rows you want to convert
- the stack expressions
- i.e. sstack(n, exp, exp2....)

In [241]:
# example first
spark.sql("""select stack(3, 1,2,3,4,5,6)""").show()

+----+----+
|col0|col1|
+----+----+
|   1|   2|
|   3|   4|
|   5|   6|
+----+----+



In [249]:
# Convert df_t into unpivoted data
# 1. reate temp view first to you can feed to sql
df_t.createOrReplaceTempView('tab')
spark.sql(""" select * from tab """).show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|  900|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



In [251]:
# notice: new colname followed by column being transposed
spark.sql(""" select dept, stack(2, 'CA_newval',CA,'NY_newval', NY ) as (state, salary) from tab""").show()

+---------+---------+------+
|     dept|    state|salary|
+---------+---------+------+
|    Sales|CA_newval|  8100|
|    Sales|NY_newval|   900|
|  Finance|CA_newval| 18900|
|  Finance|NY_newval| 16200|
|Marketing|CA_newval|  8000|
|Marketing|NY_newval|  9100|
+---------+---------+------+



In [252]:
# a better way to do that to maintain original values are
spark.sql(""" select dept, stack(2, 'CA',CA,'NY', NY ) as (state, salary) from tab""").show()

+---------+-----+------+
|     dept|state|salary|
+---------+-----+------+
|    Sales|   CA|  8100|
|    Sales|   NY|   900|
|  Finance|   CA| 18900|
|  Finance|   NY| 16200|
|Marketing|   CA|  8000|
|Marketing|   NY|  9100|
+---------+-----+------+



In [254]:
# using selectExpr
df_t.selectExpr("dept", "stack(2, 'CA',CA,'NY', NY ) as (state, salary)").show()

+---------+-----+------+
|     dept|state|salary|
+---------+-----+------+
|    Sales|   CA|  8100|
|    Sales|   NY|   900|
|  Finance|   CA| 18900|
|  Finance|   NY| 16200|
|Marketing|   CA|  8000|
|Marketing|   NY|  9100|
+---------+-----+------+

